In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
def scrape_coches_ocasion(url, diccionario):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
            print(f"El motivo REASON por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
            print(f"El motivo TEXT por el que la llamada falló es {llamada.text}")  # Imprime la razón de la falla.
            return None
    else:
        soup = BeautifulSoup(llamada.text, 'html.parser')
        coches_precios = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"}) # Sopa de coches y precios originales
        precios_descuento = soup.find_all('span', {"class":"cardVehicle_finance__SG6JV"}) # Sopa de precios con descuento
        km = soup.find_all('span', {"class":"characteristics_elements__Mb1S_"}) # Sopa de km

        for kilom in km: # Metiendo los Km en el diccionario
            if kilom.get_text('data-test').endswith('Km'):
                diccionario['km'].append(kilom.get_text('data-test'))

        for desc in precios_descuento: # Metiendo los precios con descuento en el diccionario
            if "€" in desc.text:
                diccionario['precio_rebajado'].append(desc.text)

        for nombre in coches_precios: # Metiendo los coches y precios originales en el diccionario
            if "€" in nombre.text:
                diccionario['precio_original'].append(nombre.text)
            else:
                diccionario['coche'].append(nombre.text)

        # Verificar que todas las listas tengan el mismo len
        if len(diccionario['coche']) == len(diccionario['precio_original']) == len(diccionario['precio_rebajado']) == len(diccionario['km']):
            pass
        else: # Si no tienen el mismo len, buscar el problema y corregirlo
            coches_precios_copy = [i.text for i in coches_precios]    
            counter = 0
            for i in coches_precios_copy:
                counter += 1
                if counter %2 == 0 and "€" not in i:
                    index = counter -1
                    coches_precios_copy.insert(index, "value")
                else:
                    pass
            counter = 0
            for valor in coches_precios_copy:
                counter+=1
                if valor == 'value':
                    index = counter//2-1
                    diccionario['precio_original'].insert(index, diccionario['precio_rebajado'][index])
                  
        return diccionario, pd.DataFrame(diccionario)  


url = "https://www.ocasionplus.com/coches-ocasion?v2&orderBy=morePopular"
coches = {'coche':[], 'precio_rebajado':[], 'precio_original':[], 'km':[]} # Crea diccionario para almacenar todos los datos del DataFrame
diccio, df = scrape_coches_ocasion(url, coches)


In [43]:
df

,coche,precio_rebajado,precio_original,km
0,Citroën Grand C4 Picasso,14.300€,14.300€,97.144 Km
1,Renault Mégane,14.409€,15.850€,32.875 Km
2,Toyota Rav4,15.863€,17.450€,173.793 Km
3,Toyota Yaris,8.000€,8.800€,140.848 Km
4,KIA ProCeed,21.800€,23.980€,96.100 Km
5,MG ZS,17.409€,19.150€,8.878 Km
6,Toyota C-HR,19.727€,21.700€,85.494 Km
7,Mercedes Clase C,7.500€,7.500€,113.469 Km
8,Citroën C3,9.450€,9.450€,36.779 Km
9,Audi A1,14.536€,15.990€,58.208 Km


In [48]:
len(coches['coche']) == len(coches['precio_rebajado']) == len(coches['precio_original']) == len(coches['km'])

True

In [49]:
print(len(coches['coche']))
print(len(coches['precio_rebajado']))
print(len(coches['precio_original']))
print(len(coches['km']))

20
20
20
20
